In [2]:
import numpy as np
import pandas as pd
import json
from urllib.request import urlopen
import sqlite3


# I- Load the data

In [ ]:
# question : garder  le phase quand quotation ? index à retirer ? si oui :

In [ ]:
def chunk_filtering(chunk, lst):
    template=[] #creation of an empty list :it's always cheaper to append to a list and create a DataFrame than append on a empty dataframe.
    for i in lst: 
        template.append(chunk.loc[chunk["quotation"].apply(lambda x : i in x) & 
                                  chunk["speaker"].apply(lambda x: x!= "None")].drop(['phase'], axis=1)#select the quotation with value in speaker column different from 'None' 
                                                                                #and quotations containing the key word
        
    return (pd.concat(template, ignore_index=True))# return a dataframe with our data of interest

In [ ]:
df_reader = pd.read_json('data/quotes-2020.json.bz2', lines=True, compression='bz2', chunksize=1000)
for i, chunk in enumerate(df_reader):
        chunk_clean=chunk_filtering(chunk, key_word) #recover interested row of the chunk
        header = i == 0 #we kept the name of the column only for the first chunk
        mode = 'w' if i == 0 else 'a' # For appending data to an existing CSV file (so for every chunk exepct the first one), 
                                        #we can use mode = a
            
        chunk_clean.to_csv(path_or_buf="data/clean_quotes-2020.bz2",compression='bz2',header=header, mode=mode, index = False )

### Load Quotebank data

First, let's recover the quotation of interest : as project is based on the caracterisation of the speaker, we decide to pre-select the quotations that are related to a speaker (i.e speaker value is different from 'None'). 
Moreover, we select the quotations whose subject is related to climate change : to do so we create a list of key word based on https://www.climaterealityproject.org/blog/key-terms-you-need-understand-climate-change and select quotes that contains at least one of these word.  (cf chunk_filtering method) 

In [2]:
#declaration of a key_world list
key_word = ["carbon dioxide", "greenhouse gas", "global warming",
             "climate change",  "fossil fuels", "sea-level rise",
             "renewable energy", "CO2","methane","PPM","COP","GIEC", 
             "biofuels","business as usual", "carbon footprint", "carbon neutral", "carbon sequestration"] 

In [3]:
def chunk_filtering(chunk, lst):
    template=[] #creation of an empty list :it's always cheaper to append to a list and create a DataFrame than append on a empty dataframe.
    for i in lst: 
        template.append(chunk.loc[chunk["quotation"].apply(lambda x : i in x) & 
                                  chunk["speaker"].apply(lambda x: x!= "None")].drop ()#select the quotation with value in speaker column different from 'None' 
                                                                                #and quotations containing the key word
        
    return (pd.concat(template, ignore_index=True))# return a dataframe with our data of interest

##### *2020 quotes extractions*
The original dataset is of 792,3 Mo, so we decided to divide the dataset into chucks of 1000 rows and process each of them (by using the chunck_filtering). 
Then we load the process chunck into a new csv compressed bz2 file. 

In [1]:
#maybe index = False ??

In [4]:
i=0
with pd.read_json('data/quotes-2020.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader : 
    for chunk in df_reader:
        chunk_clean=chunk_filtering(chunk, key_word) #recover interested row of the chunk
        header = i == 0 #we kept the name of the column only for the first chunk
        mode = 'w' if i == 0 else 'a' # For appending data to an existing CSV file (so for every chunk exepct the first one), 
                                        #we can use mode = a
            
        chunk_clean.to_csv(path_or_buf="data/clean_quotes-2020.bz2",compression='bz2',header=header, mode=mode)
        i+=1

In [3]:
quotes_2020= pd.read_csv('data/clean_quotes-2020.bz2', compression='bz2')

In [5]:
print( " We extract {} quotes from the 2020 files".format(len(quotes_2020)))

 We extract 11091 quotes from the 2020 files


In [7]:
quotes_2020.head()


,Unnamed: 0,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,0,2020-02-24-013709,For every doubling of carbon dioxide concentra...,E. Calvin Beisner,['Q19877395'],2020-02-24 16:02:23,1,"[['E. Calvin Beisner', '0.677'], ['None', '0.3...",['https://www.heartland.org/news-opinion/news/...,E
1,1,2020-02-24-028340,If you're a doctor that cares about the wellbe...,Fiona Stanley,['Q1653736'],2020-02-24 12:45:00,4,"[['Fiona Stanley', '0.9473'], ['None', '0.0527']]",['http://watoday.com.au/business/banking-and-f...,E
2,2,2020-01-24-004182,"Also, spoke about a range of emerging sectors ...",Piyush Goyal,['Q7199798'],2020-01-24 19:02:14,2,"[['Piyush Goyal', '0.6385'], ['Peter Voser', '...",['http://aninews.in/news/world/europe/piyush-g...,E
3,0,2020-01-29-062975,Many make the link today between their experie...,Peter Maurer,"['Q117796', 'Q42426597']",2020-01-29 09:04:36,5,"[['Peter Maurer', '0.8787'], ['None', '0.1213']]",['http://whbl.com/news/articles/2020/jan/29/hu...,E
4,1,2020-02-10-076321,the National Energy and Climate Plans are how ...,Kadri Simson,['Q13570003'],2020-02-10 05:51:51,1,"[['Kadri Simson', '0.9269'], ['None', '0.0504'...",['https://www.politico.eu/newsletter/brussels-...,E


##### *2019 quotes extractions*
The original dataset is of 3.32 Go, so we decided to divide the dataset into chucks of 1000 rows and process each of them (by using the chunck_filtering). Then we load the process chunck into a new csv compressed bz2 file.

In [9]:
i=0
with pd.read_json('data/quotes-2019.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        chunk_clean=chunk_filtering(chunk, key_word) #recover interested row of the chunk
        header = i == 0 #we kept the name of the column only for the first chunk
        mode = 'w' if i == 0 else 'a' # For appending data to an existing CSV file (so for every chunk exepct the first one), 
                                        #we can use mode = a
            
        chunk_clean.to_csv(path_or_buf='data/clean_quotes-2019.bz2', compression='bz2', mode = mode, header=header) # create a new csv files compress with bz2 containing all the dataframe recover from the chunk; 
        i+=1  

In [6]:
quotes_2019= pd.read_csv('data/clean_quotes-2019.bz2', compression='bz2') # load into the quotes_2019 df

FileNotFoundError: [Errno 2] No such file or directory: 'data/clean_quotes-2019.bz2'

In [ ]:
print( " We extracted {} quotes from the 2019 files".format(len(quotes_2019)))

(47280, 10)

##### *2018 quotes extractions*
The original dataset is of 4.48 Go, so we decided to divide the dataset into chucks of 1000 rows and process each of them ((by using the chunck_filtering). Then we load the process chunck into a new csv compressed bz2 file.

In [14]:
i=0
with pd.read_json('data/quotes-2018.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        
        chunk_clean=chunk_filtering(chunk, key_word) #recover interested row of the chunk
        header = i == 0 #we kept the name of the column only for the first chunk
        mode = 'w' if i == 0 else 'a' # For appending data to an existing CSV file (so for every chunk exepct the first one), 
                                        #we can use mode = a
            
        chunk_clean.to_csv(path_or_buf='data/clean_quotes-2018.bz2', compression='bz2', mode = mode, header=header)
        i+=1

In [7]:
quotes_2018= pd.read_csv('data/clean_quotes-2018.bz2', compression='bz2') #load the data to quotes_2018 df

In [18]:
print( " We extracted {} quotes from the 2018 files".format(len(quotes_2018)))

 We extracted 35847 quotes from the 2018 files


##### *2017 quotes extractions*
The original dataset is of 4.84 Go, so we decided to divide the dataset into chucks of 1000 rows and process each of them (by using the chunck_filtering). Then we load the process chunck into a new csv compressed bz2 file

In [17]:
i=0
with pd.read_json('data/quotes-2017.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        chunk_clean=chunk_filtering(chunk, key_word) #recover interested row of the chunk
        header = i == 0 #we kept the name of the column only for the first chunk
        mode = 'w' if i == 0 else 'a' # For appending data to an existing CSV file (so for every chunk exepct the first one), 
                                        #we can use mode = a
            
        chunk_clean.to_csv(path_or_buf='data/clean_quotes-2017.bz2', compression='bz2', mode = 'a') # create a new csv files compress with bz2 containing all the dataframe recover from the chunk;
        i+=1

In [11]:
quotes_2017= pd.read_csv('data/clean_quotes-2017.bz2', compression='bz2')

In [16]:
print( " We extracted {} quotes from the 2017 files".format(len(quotes_2017)))

 We extracted 35324 quotes from the 2017 files


##### *2016 quotes extractions*
The original dataset is of 2.16 Go, so we decided to divide the dataset into chucks of 1000 rows and process each of them(by using the chunck_filtering). Then we load the process chunck into a new csv compressed bz2 file.

In [20]:
i=0
with pd.read_json('data/quotes-2016.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        chunk_clean=chunk_filtering(chunk, key_word) #recover interested row of the chunk
        header = i == 0 #we kept the name of the column only for the first chunk
        mode = 'w' if i == 0 else 'a' # For appending data to an existing CSV file (so for every chunk exepct the first one), 
                                        #we can use mode = a
            
        chunk_clean.to_csv(path_or_buf='data/clean_quotes-2016.bz2', compression='bz2', mode = 'a') # create a new csv files compress with bz2 containing all the dataframe recover from the chunk;
        i+=1

In [14]:
quotes_2016= pd.read_csv('data/clean_quotes-2016.bz2', compression='bz2')

In [17]:
print( " We extracted {} quotes from the 2016 files".format(len(quotes_2016)))

 We extracted 18344 quotes from the 2016 files


##### *2015 quotes extractions*
The original dataset is of 3.11 Go, so we decided to divide the dataset into chucks of 1000 rows and process each of them(by using the chunck_filtering). Then we load the process chunck into a new csv compressed bz2 file.

In [4]:
i=0
with pd.read_json('data/quotes-2015.json.bz2', lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        chunk_clean=chunk_filtering(chunk, key_word) #recover interested row of the chunk
        header = i == 0 #we kept the name of the column only for the first chunk
        mode = 'w' if i == 0 else 'a' # For appending data to an existing CSV file (so for every chunk exepct the first one), 
                                        #we can use mode = a
            
        chunk_clean.to_csv(path_or_buf='data/clean_quotes-2015.bz2', compression='bz2', mode = 'a') # create a new csv files compress with bz2 containing all the dataframe recover from the chunk;
        i+=1

(13829, 10)

In [19]:
quotes_2015= pd.read_csv('data/clean_quotes-2015.bz2', compression='bz2')

In [20]:
print( " We extracted {} quotes from the 2015 files".format(len(quotes_2015)))

 We extracted 35176 quotes from the 2015 files


In [109]:
print(" At result, we extracted {} quotes fromes quotebank data".format((len(quotes_2015)+len(quotes_2016)+len(quotes_2017)
                                                                         +len(quotes_2018)+len(quotes_2019)+len(quotes_2020)))

Now that we extracted the interesting data from the Quotebank data, let's add another dataset that will give us characteristic information about the speaker

## Load additional data Relative to speakers

The provided speaker_attributes.parquet file contains attributes in terms of QIDs, thereby being uninterpretable by humans (df_qid).
To map the QIDs to meaningful labels, we used the provied wikidata_labels_descriptions_quotebank.csv.bz2 containg the labels and value fo the respective QID containing the df_qid (df_label_qid)
By combaning the information of both we can obtained usefule information about speakers. 

In [22]:
df_qid = pd.read_parquet("speaker_attributes.parquet",engine= "pyarrow" )
df_label_qid = pd.read_csv('data/wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')

Before extract the label of qid, let's check which column we want to keep in frame with our project

In [72]:
df_qid.head(3)

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None


Let's verify that academic_degree has revelant values

In [90]:
print("There's no academic degree revelant value ? {}".format(all(df_qid.academic_degree.isna())))

There's no academic degree revelant value ? False


We decided to drop lastrevid, US_congress_bio_ID, type. Moreover, it's seems that academic_degree value are rare, let's check that too.

In [88]:
df_qid.drop(['lastrevid', 'US_congress_bio_ID', 'type'], axis=1, inplace=True)

,aliases,date_of_birth,nationality,gender,ethnic_group,occupation,party,academic_degree,id,label,candidacy,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],None,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],[Q7994501],"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]","[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],None,[Q1028181],None,None,Q297,Diego Velázquez,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
9055976,[Barker Howard],None,[Q30],[Q6581097],None,[Q82955],None,None,Q106406560,Barker B. Howard,None,None
9055977,[Charles Macomber],None,[Q30],[Q6581097],None,[Q82955],None,None,Q106406571,Charles H. Macomber,None,None
9055978,None,[+1848-04-01T00:00:00Z],None,[Q6581072],None,None,None,None,Q106406588,Dina David,None,None
9055979,None,[+1899-03-18T00:00:00Z],None,[Q6581072],None,None,None,None,Q106406593,Irma Dexinger,None,None


In [23]:
df_label_qid.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


In [23]:
#We found out that some of the QIDs used in the speaker attribute file are actually redirection from an original QID. 
#We will manulally add their corresponding information using the orginal QID. We found the corespondance manualy between the two. 
#Here, there are in order, respectively the redirection QID, and its corresponding original one. One of he QID was only present 
#as a redirection, so we manually added this one (Q3186984), and its corresponding info. 

redirect_QID=['Q3268166', 'Q11815360', 'Q12014399', 'Q16287483',
              'Q20432251', 'Q21550646', 'Q13365117', 'Q13424794',
             'Q1248362', 'Q6859927', 'Q15145782',
             'Q15991263', 'Q12455619', 'Q5568256', 
             'Q6363085', 'Q11819457', 'Q12334852', 'Q15145783']
actual_QID=['Q1113899', 'Q1919436', 'Q250867', 'Q6051619',
             'Q26934816', 'Q18431816', 'Q12840545', 'Q5157338',
            'Q3455803', 'Q715222', 'Q1052281',
            'Q2743689', 'Q7019111', 'Q3738699', 
            'Q380075', 'Q3391743', 'Q476246', 'Q2449503']

#There is a QID that was deleted from Wikidata, Q99753484, so we will remove this QID later 

lst=[['Journalist', 'monthly magazine of the United Kingdom‘s National Union of Journalists (NUJ)']]
indexes=['Q3186984']
col=['Label', 'Description']
for i in range(len(redirect_QID)):
    lst.append([df_label_qid.loc[actual_QID[i]]['Label'], 
                df_label_qid.loc[actual_QID[i]]['Description']])
    indexes.append(redirect_QID[i])

additional_df= pd.DataFrame(lst, columns= col, index=indexes)
df_label_qid_co=df_label_qid.append(additional_df, ignore_index= False)

In [25]:
#Function that loops through a column to replace QID with their label, and skip None values. We will deal with them later, 
#when we will use the different data.
def l(df) : 
    liste=[]
    for row  in df: 
        if row is None: 
            continue #skip None values
        template=[]
        for value in row: #iterating over the values of a cell, as there are multiple QIDs in some of them.
            if value == 'Q99753484': #To filter the deleted QID
                continue
            template.append(df_label_qid_co.loc[value]['Label']) #Map the QID to its corresponding label. 
        liste.append(template)
    return pd.Series(liste)
#Applying the function to every column containing QIDs. 
df_qid [['nationality', 'gender', 'ethnic_group','occupation', 'party', 'academic_degree', 'candidacy', 'religion']] = df_qid[['nationality', 'gender', 'ethnic_group','occupation', 'party', 'academic_degree', 'candidacy', 'religion']].apply(l)



# II- Filter the data

As a good data scientist, the first thing to do is to clean up the data : we need to filtered missing and duplicates rows if there are presented.

*check for missing row*

In [51]:
print("Is there some missing rows ? {} ".format(np.array([quotes_2020.isnull().any(axis=1)]).all()))

Is there some missing rows ? False 


##### *check for duplicate*

We define a function that receive a dataframe (quotes_2020 ... quotes_2015) and remove their duplicates rows according to duplicate quotation;

In [57]:
def check_duplicates (df): 
    
    if df["quotation"].is_unique == False : 
        df.drop_duplicates(['quotation'], keep='first', inplace=True) #remove the duplicate rows directly on the df 

In [59]:
check_duplicates(quotes_2020)
check_duplicates(quotes_2019)
check_duplicates(quotes_2018)
check_duplicates(quotes_2017)
check_duplicates(quotes_2016)
check_duplicates(quotes_2017)
check_duplicates(quotes_2015)
print( "We still get {} quotes from the 2020 dataset".format(len(quotes_2020)))
print( "We still get {} quotes from the 2020 dataset".format(len(quotes_2019)))
print( "We still get {} quotes from the 2020 dataset".format(len(quotes_2018)))
print( "We still get {} quotes from the 2020 dataset".format(len(quotes_2017)))
print( "We still get {} quotes from the 2020 dataset".format(len(quotes_2016)))
print( "We still get {} quotes from the 2020 dataset".format(len(quotes_2015)))

We still get 10486 quotes from the 2020 dataset


### Merge quotebank extracted data and caracteristics of the speakers

In [92]:
#test for 2020

In [91]:
quotes_2020.merge(df_qid, left_on = 'speaker' ,right_on = 'label' , how = 'inner')

,Unnamed: 0,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,...,nationality,gender,ethnic_group,occupation,party,academic_degree,id,label,candidacy,religion
0,0,2020-02-24-013709,For every doubling of carbon dioxide concentra...,E. Calvin Beisner,['Q19877395'],2020-02-24 16:02:23,1,"[['E. Calvin Beisner', '0.677'], ['None', '0.3...",['https://www.heartland.org/news-opinion/news/...,E,...,[Q30],[Q6581097],None,[Q1607826],None,None,Q19877395,E. Calvin Beisner,None,None
1,0,2020-02-24-010293,Crops also improve their fruit-to-fiber ratio ...,E. Calvin Beisner,['Q19877395'],2020-02-24 16:02:23,1,"[['E. Calvin Beisner', '0.7538'], ['None', '0....",['https://www.heartland.org/news-opinion/news/...,E,...,[Q30],[Q6581097],None,[Q1607826],None,None,Q19877395,E. Calvin Beisner,None,None
2,1,2020-02-24-028340,If you're a doctor that cares about the wellbe...,Fiona Stanley,['Q1653736'],2020-02-24 12:45:00,4,"[['Fiona Stanley', '0.9473'], ['None', '0.0527']]",['http://watoday.com.au/business/banking-and-f...,E,...,[Q408],[Q6581072],None,[Q13416803],None,None,Q1653736,Fiona Stanley,None,[Q6423963]
3,2,2020-01-24-004182,"Also, spoke about a range of emerging sectors ...",Piyush Goyal,['Q7199798'],2020-01-24 19:02:14,2,"[['Piyush Goyal', '0.6385'], ['Peter Voser', '...",['http://aninews.in/news/world/europe/piyush-g...,E,...,[Q668],[Q6581097],None,[Q82955],[Q10230],None,Q7199798,Piyush Goyal,None,[Q9089]
4,0,2020-01-21-092711,We are like a pivot for the Indian Ocean and w...,Piyush Goyal,['Q7199798'],2020-01-21 13:07:20,1,"[['Piyush Goyal', '0.9241'], ['None', '0.0759']]",['https://www.newindianexpress.com/world/2020/...,E,...,[Q668],[Q6581097],None,[Q82955],[Q10230],None,Q7199798,Piyush Goyal,None,[Q9089]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46645,1,2020-02-21-082953,Where these sorts of things can be very useful...,David Farrell,"['Q18921713', 'Q21931971', 'Q46829026', 'Q5233...",2020-02-21 21:38:09,1,"[['David Farrell', '0.8397'], ['None', '0.1603']]",['https://inews.co.uk/news/environment/climate...,E,...,"[Q27, Q145]",[Q6581097],None,[Q1238570],None,None,Q18921713,David Farrell,None,None
46646,0,2020-01-07-040598,It will simulate adverse weather conditions; i...,Satoshi Matsuoka,['Q11530057'],2020-01-07 13:03:19,1,"[['Satoshi Matsuoka', '0.8561'], ['None', '0.1...",['http://www.sciencebusiness.net/international...,E,...,None,None,None,[Q1650915],None,None,Q89726452,Satoshi Matsuoka,None,None
46647,0,2020-01-07-040598,It will simulate adverse weather conditions; i...,Satoshi Matsuoka,['Q11530057'],2020-01-07 13:03:19,1,"[['Satoshi Matsuoka', '0.8561'], ['None', '0.1...",['http://www.sciencebusiness.net/international...,E,...,None,[Q6581097],None,[Q82594],None,None,Q11530057,Satoshi Matsuoka,None,None
46648,0,2020-01-07-040598,It will simulate adverse weather conditions; i...,Satoshi Matsuoka,['Q11530057'],2020-01-07 13:03:19,1,"[['Satoshi Matsuoka', '0.8561'], ['None', '0.1...",['http://www.sciencebusiness.net/international...,E,...,None,None,None,[Q1650915],None,None,Q99365468,Satoshi Matsuoka,None,None


## I-Exploration of our data

Let's see some distribution and statitics: 
 - aged people vs yound people 
 - party politics  
ect... 

stat : correlation coeff ; m